In [1]:
import pandas as pd
import numpy as np
import sympy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import warnings
warnings.filterwarnings('ignore')

In [2]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(object)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [3]:
def demand_model(df):
    import pandas as pd
    import numpy as np 
  
    def demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
        D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
        return D/(1+D)

   
    return  demanda(df['premio'], df['is_casco_ofer'], df['GBM_mercado'], df['Premio_IC_100'], df['premio_anterior'], df['MD_tempo_casa_2'], df['per_com'], df['per_com_bin_p_1_10'], df['canal_bin_Corretor_Mais'], df['Cod_End_bin_13509'], df['fator_ajuste_bin_f_101_120'], df['classe_bonus_bin_00'], df['UF_bin_AL_CE_RS'], df['UF_bin_AP_DF_ES_PA_PB_RN_SC'], df['UF_bin_GO_MA_MG'], df['UF_bin_MT_PR'], df['UF_bin_RJ'], df['MD_comb_bin_Gasolina'], df['md_tipo_renov_ant_bin_1_SeguroNovo'], df['md_tipo_renov_ant_bin_3_RenovCongenere'], df['fx_ir_12m_bin_01_00_20'], df['fx_ir_12m_bin_03_40_60'], df['fx_ir_12m_bin_04_60_80'], df['fx_ir_12m_bin_05_80_100'], df['MD_tipo_renov_bin_3_RenovPropria_com_sin'], df['idade_ult_RNS_bin_01_05_anos'], df['idade_ult_RNS_bin_06_inf'])

In [4]:
def relatorio(df):
    import pandas as pd
    import numpy as np 
    
    df["tarifa"] = df["premio"]
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["Renewal_Demand_basecase"]*df["premio"]
    df["tarifa_prob"] = df["Renewal_Demand_basecase"]*df["tarifa"]
    df["premio_anterior_prob"] = df["Renewal_Demand_basecase"]*df["premio_anterior"]
    df["Act_cost_prob"] = df["Renewal_Demand_basecase"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["premio"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]


    tarifa              = np.mean(df["tarifa"])
    tarifa_w            = np.sum(df["tarifa_prob"])/np.sum(df["Renewal_Demand_basecase"])
    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["Renewal_Demand_basecase"]),0)
    premio_ofertado     = np.mean(df["premio"])
    retencao_estimada   = np.mean(df["Renewal_Demand_basecase"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["Renewal_Demand_basecase"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["Renewal_Demand_basecase"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["premio"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["premio"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["Renewal_Demand_basecase"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["Renewal_Demand_basecase"])

    Relative_Core_Premium_Increase_Offered = (premio_ofertado-premio_anterior)/premio_anterior
    Relative_Core_Premium_Increase_Written = (premio_anterior-premio_anterior_w)/premio_anterior_w
    Relative_Core_Optimization_Increase_Offered=(premio_ofertado-tarifa)/tarifa
    Relative_Core_Optimization_Increase_Written=(premio_ofertado-tarifa)/premio_anterior_w
    Core_Premium_Increase_Offered=(premio_ofertado-premio_anterior)
    Core_Premium_Increase_Written=(premio_esperado-premio_anterior_w)


    tabela = pd.DataFrame([apolice_ofertada,apolice_estimada,
                            retencao_estimada,
                            premio_ofertado,premio_esperado,
                            premio_anterior,premio_anterior_w,
                            Relative_Core_Premium_Increase_Offered,
                            Relative_Core_Premium_Increase_Written,
                            Relative_Core_Optimization_Increase_Offered,
                            Relative_Core_Optimization_Increase_Written,
                            Core_Premium_Increase_Offered,
                            Core_Premium_Increase_Written,
                            risco_ofertado,risco_esperado,
                            comis_ofertado, comis_esperado,
                            sp_ofertado,sp_esperado,
                            ic_ofertado,ic_esperado                  
                          ])



    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Relative_Core_Premium_Increase_Offered:',
                    'Relative_Core_Premium_Increase_Written:',
                    'Relative_Core_Optimization_Increase_Offered',
                    'Relative_Core_Optimization_Increase_Written:',
                    'Core_Premium_Increase_Offered:',
                    'Core_Premium_Increase_Written:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:'
                    ]
    return tabela

In [5]:
def relatorio_otim(df):
    import pandas as pd
    import numpy as np 
    
    df["tarifa"] = df["premio"]
    #df["retencao_otimizada"] = df["retencao_otimizada"]
    #df["precos_otimizados"] = df["precos_otimizados"]
    
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["retencao_otimizada"]*df["precos_otimizados"]
    df["premio_anterior_prob"] = df["retencao_otimizada"]*df["premio_anterior"]
    df["tarifa_prob"] = df["retencao_otimizada"]*df["tarifa"]
    df["Act_cost_prob"] = df["retencao_otimizada"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["precos_otimizados"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]


    tarifa              = np.mean(df["tarifa"])
    tarifa_w            = np.sum(df["tarifa_prob"])/np.sum(df["retencao_otimizada"])
    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["retencao_otimizada"]),0)
    premio_ofertado     = np.mean(df["precos_otimizados"])
    retencao_estimada   = np.mean(df["retencao_otimizada"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["retencao_otimizada"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["retencao_otimizada"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["precos_otimizados"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["precos_otimizados"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["retencao_otimizada"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["retencao_otimizada"])

    Relative_Core_Premium_Increase_Offered = (premio_ofertado-premio_anterior)/premio_anterior
    Relative_Core_Premium_Increase_Written = (premio_anterior-premio_anterior_w)/premio_anterior_w
    Relative_Core_Optimization_Increase_Offered=(premio_ofertado-tarifa)/tarifa
    Relative_Core_Optimization_Increase_Written=(premio_ofertado-tarifa)/premio_anterior_w
    Core_Premium_Increase_Offered=(premio_ofertado-premio_anterior)
    Core_Premium_Increase_Written=(premio_esperado-premio_anterior_w)

    
    
    
    tabela = pd.DataFrame([round(apolice_ofertada,0),
                            round(apolice_estimada,0),
                            round(retencao_estimada,3),
                            round(premio_ofertado,2),
                            round(premio_esperado,2),
                            round(premio_anterior,2),
                            round(premio_anterior_w,2),
                            round(Relative_Core_Premium_Increase_Offered,2),
                            round(Relative_Core_Premium_Increase_Written,2),
                            round(Relative_Core_Optimization_Increase_Offered,2),
                            round(Relative_Core_Optimization_Increase_Written,2),
                            round(Core_Premium_Increase_Offered,2),
                            round(Core_Premium_Increase_Written,2),
                            round(risco_ofertado,2),
                            round(risco_esperado,2),
                            round(comis_ofertado,2),
                            round(comis_esperado,2),
                            round(sp_ofertado,2),
                            round(sp_esperado,4),
                            round(ic_ofertado,4),
                            round(ic_esperado,4)      
                            
                            ])


    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Relative_Core_Premium_Increase_Offered:',
                    'Relative_Core_Premium_Increase_Written:',
                    'Relative_Core_Optimization_Increase_Offered:',
                    'Relative_Core_Optimization_Increase_Written:',
                    'Core_Premium_Increase_Offered:',
                    'Core_Premium_Increase_Written:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:'
                    ]

    return tabela

In [6]:
def elasticidade(df):
    
    df['tarifa']=df['premio']
    df['numbers'] = df['premio']/df['tarifa']
    if df['numbers'].any()>1 :
        df['X27'] = numbers-1
    else:
         df['X27']=1
    
    
    P=(df['premio'])
    X1=np.array(df['is_casco_ofer'])
    X2=np.array(df['GBM_mercado'])
    X3=np.array(df['Premio_IC_100'])
    X4=np.array(df['premio_anterior'])
    X5=np.array(df['MD_tempo_casa_2'])
    X6=np.array(df['per_com'])
    X7=np.array(df['per_com_bin_p_1_10'])
    X8=np.array(df['canal_bin_Corretor_Mais'])
    X9=np.array(df['Cod_End_bin_13509'])
    X10=np.array(df['fator_ajuste_bin_f_101_120'])
    X11=np.array(df['classe_bonus_bin_00'])
    X12=np.array(df['UF_bin_AL_CE_RS'])
    X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14=np.array(df['UF_bin_GO_MA_MG'])
    X15=np.array(df['UF_bin_MT_PR'])
    X16=np.array(df['UF_bin_RJ'])
    X17=np.array(df['MD_comb_bin_Gasolina'])
    X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20=np.array(df['fx_ir_12m_bin_01_00_20'])
    X21=np.array(df['fx_ir_12m_bin_03_40_60'])
    X22=np.array(df['fx_ir_12m_bin_04_60_80'])
    X23=np.array(df['fx_ir_12m_bin_05_80_100'])
    X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X27=np.array(df['X27'])

    
    
    
    # Definindo as variáveis simbólicas
    P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, X27 = sp.symbols('P X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15 X16 X17 X18 X19 X20 X21 X22 X23 X24 X25 X26 X27')

    # Definindo a função de demanda
    demanda = sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26)/(1+sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26 -3*X27))
    
    # Derivada da demanda em relação ao preço
    demanda_derivada_P = sp.diff(demanda, P)

    # Funções lambdify para avaliação numérica
    demanda_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda, 'numpy')
    demanda_derivada_P_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda_derivada_P, 'numpy')

    
    # Extraindo os valores das variáveis
    lista_variaveis_na_ordem = ['is_casco_ofer','GBM_mercado','Premio_IC_100','premio_anterior','MD_tempo_casa_2','per_com','per_com_bin_p_1_10','canal_bin_Corretor_Mais','Cod_End_bin_13509','fator_ajuste_bin_f_101_120','classe_bonus_bin_00','UF_bin_AL_CE_RS','UF_bin_AP_DF_ES_PA_PB_RN_SC','UF_bin_GO_MA_MG','UF_bin_MT_PR','UF_bin_RJ','MD_comb_bin_Gasolina','md_tipo_renov_ant_bin_1_SeguroNovo','md_tipo_renov_ant_bin_3_RenovCongenere','fx_ir_12m_bin_01_00_20','fx_ir_12m_bin_03_40_60','fx_ir_12m_bin_04_60_80','fx_ir_12m_bin_05_80_100','MD_tipo_renov_bin_3_RenovPropria_com_sin','idade_ult_RNS_bin_01_05_anos','idade_ult_RNS_bin_06_inf','X27']

    X_vals = np.array(df.loc[:, lista_variaveis_na_ordem])
    P_val = df['premio'].values

    # Calculando os valores de demanda e sua derivada
    D_val = demanda_func(P_val, *X_vals.T)
    dD_dP_val = demanda_derivada_P_func(P_val, *X_vals.T)

    # Calculando a elasticidade-preço da demanda
    elasticidade_preco = (dD_dP_val * P_val) / D_val
    return elasticidade_preco

## Base Para escorar e otimizar

In [7]:
df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/AJUSTADO Base Case Earnix - Grupo Segmentacao.csv",
                             dtype={'chave': 'string'},
                     encoding='latin-1',
                     delimiter='|')

df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

In [8]:
df_lote_escorar=df_lote_102024.copy()

In [9]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [10]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [11]:
df_lote = categorizar(df_lote_escorar)

In [12]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [13]:
df_lote['Renewal_Demand_basecase'] = demand_model(df_lote)

In [14]:
df_lote['elasticidade']=elasticidade(df_lote)

In [15]:
resumo_basecase = relatorio(df_lote)
resumo_basecase

,Total
Apolices Ofertadas:,1473.000000
Apolices Renovadas:,756.000000
%Retenção:,0.513468
Premio Ofertado:,3063.388364
Premio Renovado:,2381.388210
Premio Anterior Ofertado:,2893.832077
Premio Anterior Renovado:,2614.777661
Relative_Core_Premium_Increase_Offered:,0.058592
Relative_Core_Premium_Increase_Written:,0.106722
Relative_Core_Optimization_Increase_Offered,0.000000


## Otimização  

In [16]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

(1473, 187)

In [17]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

(1473, 187)

#########################################

In [18]:
import time
def timeit(price):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = price(*args, **kwargs)
        end = time.perf_counter()
        elapsed = (end - start)/60
        print(f'Tempo: {elapsed:.6f} Minutos')
        return result
    return wrapper

@timeit




def price(df,fator_premio=1.23):
   
    ######
    import sympy as sp
    from scipy.optimize import fsolve


    P=(df['premio'])
    X1=np.array(df['is_casco_ofer'])
    X2=np.array(df['GBM_mercado'])
    X3=np.array(df['Premio_IC_100'])
    X4=np.array(df['premio_anterior'])
    X5=np.array(df['MD_tempo_casa_2'])
    X6=np.array(df['per_com'])
    X7=np.array(df['per_com_bin_p_1_10'])
    X8=np.array(df['canal_bin_Corretor_Mais'])
    X9=np.array(df['Cod_End_bin_13509'])
    X10=np.array(df['fator_ajuste_bin_f_101_120'])
    X11=np.array(df['classe_bonus_bin_00'])
    X12=np.array(df['UF_bin_AL_CE_RS'])
    X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14=np.array(df['UF_bin_GO_MA_MG'])
    X15=np.array(df['UF_bin_MT_PR'])
    X16=np.array(df['UF_bin_RJ'])
    X17=np.array(df['MD_comb_bin_Gasolina'])
    X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20=np.array(df['fx_ir_12m_bin_01_00_20'])
    X21=np.array(df['fx_ir_12m_bin_03_40_60'])
    X22=np.array(df['fx_ir_12m_bin_04_60_80'])
    X23=np.array(df['fx_ir_12m_bin_05_80_100'])
    X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X27=np.array(df['X27'])


# função demanda
    def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
        D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
        return D/(1+D)
    
    
    def ajustar_preco(custo, elasticidade):
        
    # Ajuste de preço baseado na elasticidade
        novo_preco = custo / (1 + 1 / elasticidade)
        return novo_preco 

    
    
# função objetivo 
    def objetivo(P):

        df['precos']= P *fator_premio

        df['tarifa']=df['premio']

        df['numbers'] = df['precos']/df['tarifa']

        if (df['numbers']>1).any():
            df['X27'] = df['numbers']-1
        else:
             df['X27']=0

        df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.6,-3.55,-3.5,-3.45,-3.4,-3.35,-3.3,-3.25,-3.2,-3.15,-3.1,-3.05,-3,-2.95,-2.9,-2.85,-2.8,-2.75,-2.7,-2.65,-2.6,-2.55,-2.5,-2.45,-2.4,-2.35,-2.3,-2.25,-2.2,-2.15,-2.10,0])  
        df['elasticidade_abs'] = abs(df['elasticidade'])
        df['elasticidade_abs_bin'] = pd.cut(x=df['elasticidade_abs'], bins=[0,2.1,2.15,2.2,2.25,2.3,2.35,2.4,2.45,2.5,2.55,2.6,2.65,2.7,2.75,2.8,2.85,2.9,2.95,3,3.05,3.1,3.15,3.2,3.25,3.3,3.35,3.4,3.45,3.5,3.55,3.6])  


      
        
# Calcular a retenção 
        df['retencao_otimizada1'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

# Calcular a margem esperada 
        df['margem_esperada_otimizada'] =   df['retencao_otimizada1']*(df['precos'] - df['Act_cost'])
        margem_esperada_total = df['margem_esperada_otimizada'].sum() 
        return -margem_esperada_total

# Definir os limites para os preços otimizado conforme o Earnix
    nome_variaveis = ['LB','UB']
    limites=np.array(df.loc[:,nome_variaveis])

# Executar a otimização 
    resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=11,seed=df.shape[0])

# Resultados
    precos_otimizados = resultado.x
    margem_esperada_total_otimizada = -resultado.fun

    df['precos_otimizados'] = precos_otimizados
    df['retencao_otimizada'] = calcular_retencao(df['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

# validação
    df['batente_otim']=np.round(df['precos_otimizados']/df['premio'],2)
    df['batente_otim_prem_ant']=np.round(df['precos_otimizados']/df['premio_anterior'],2)

    df.drop(['precos','retencao_otimizada1'],axis='columns', inplace=True)
    
    print(resultado.message) 
    print(relatorio_otim(df))
    return df

In [19]:
Lote_102024_v1 =price(df_lote_otim,fator_premio=1.23)

Optimization terminated successfully.
                                                  Total
Apolices Ofertadas:                           1473.0000
Apolices Renovadas:                            750.0000
%Retenção:                                       0.5090
Premio Ofertado:                              3065.5000
Premio Renovado:                              2408.0300
Premio Anterior Ofertado:                     2893.8300
Premio Anterior Renovado:                     2615.1800
Relative_Core_Premium_Increase_Offered:          0.0600
Relative_Core_Premium_Increase_Written:          0.1100
Relative_Core_Optimization_Increase_Offered:     0.0000
Relative_Core_Optimization_Increase_Written:     0.0000
Core_Premium_Increase_Offered:                 171.6700
Core_Premium_Increase_Written:                -207.1400
Risco Ofertado                                1767.8100
Risco Renovado                                1353.9500
%Comissão Ofertada:                              0.1600
%Comissão 

In [52]:
#Validação dos batentes
Lote_102024_v1[['batente_otim_prem_ant','batente_otim','elasticidade','elasticidade_abs','precos_otimizados','retencao_otimizada',]].describe()

,batente_otim_prem_ant,batente_otim,elasticidade,elasticidade_abs,precos_otimizados,retencao_otimizada
count,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000
mean,1.068853,1.006572,-3.216151,3.216151,3065.500666,0.509464
std,0.354766,0.051566,0.388735,0.388735,2241.625157,0.258597
min,0.520000,0.900000,-3.563048,1.029459,491.781988,0.013957
25%,0.780000,0.970000,-3.456063,3.149575,1689.852360,0.300120
50%,0.950000,1.000000,-3.349988,3.349988,2447.620169,0.532095
75%,1.270000,1.050000,-3.149575,3.456063,3633.257575,0.715785
max,1.800000,1.100000,-1.029459,3.563048,21192.498000,0.984617


## Exporting the Data frame.

In [54]:
analitico_202410_ajustado=Lote_102024_v1
analitico_202410_ajustado.to_csv(r'analitico_202410_ajustado_elaticidade2.csv', sep='|' ,header='true',index=False)

In [284]:
def fator_elasticidade(df):   
    df['Fator_elasticidade'] = df['elasticidade_abs'].apply(lambda x: 1.03   if x <= 2.95 else 
                                                                      1.01   if x <= 3.3 else 
                                                                      0.97   if x <=  3.55 else 1.02)   
    return df  

In [285]:
Lote_102024_v1 = fator_elasticidade(Lote_102024_v1)

In [286]:
Lote_102024_v1[['batente_otim_prem_ant','batente_otim','elasticidade','Fator_elasticidade']].describe()

,batente_otim_prem_ant,batente_otim,elasticidade,Fator_elasticidade
count,1473.000000,1473.000000,1473.000000,1473.000000
mean,1.064637,1.006891,-3.216151,0.989552
std,0.340492,0.063508,0.388735,0.024262
min,0.500000,0.880000,-3.563048,0.970000
25%,0.800000,0.960000,-3.456063,0.970000
50%,0.950000,1.000000,-3.349988,0.970000
75%,1.260000,1.050000,-3.149575,1.010000
max,1.840000,1.170000,-1.029459,1.030000


In [287]:
Lote_102024_v11=Lote_102024_v1
Lote_102024_v11['precos_otimizados2'] = Lote_102024_v11['precos_otimizados']*Lote_102024_v11['Fator_elasticidade']
Lote_102024_v11['retencao_otimizada2'] = calcular_retencao(Lote_102024_v11['precos_otimizados2'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)


In [288]:
Lote_102024_v11[['precos_otimizados','precos_otimizados2','retencao_otimizada','retencao_otimizada2']].describe()

,precos_otimizados,precos_otimizados2,retencao_otimizada,retencao_otimizada2
count,1473.000000,1473.000000,1473.000000,1473.000000
mean,3041.341802,2989.312835,0.512115,0.520733
std,2186.319008,2113.577882,0.249175,0.240460
min,521.288907,536.927574,0.014962,0.016614
25%,1708.939945,1705.486289,0.315399,0.339170
50%,2417.754644,2368.238996,0.546877,0.567637
75%,3596.161981,3535.032098,0.708470,0.703519
max,20768.648040,20145.588599,0.981144,0.979101


In [289]:

Lote_102024_v11['precos_otimizados']  = Lote_102024_v11['precos_otimizados2'] 
Lote_102024_v11['retencao_otimizada']  = Lote_102024_v11['retencao_otimizada2'] 

Lote_102024_v11['batente_otim']=np.round(Lote_102024_v11['precos_otimizados']/df['premio'],2)
Lote_102024_v11['batente_otim_prem_ant']=np.round(Lote_102024_v11['precos_otimizados']/df['premio_anterior'],2)


In [290]:
Lote_102024_v11[['batente_otim_prem_ant','batente_otim','elasticidade','Fator_elasticidade']].describe()

,batente_otim_prem_ant,batente_otim,elasticidade,Fator_elasticidade
count,1473.000000,1473.000000,1473.000000,1473.000000
mean,1.050652,0.997183,-3.216151,0.989552
std,0.325669,0.080340,0.388735,0.024262
min,0.490000,0.860000,-3.563048,0.970000
25%,0.800000,0.940000,-3.456063,0.970000
50%,0.950000,0.980000,-3.349988,0.970000
75%,1.230000,1.050000,-3.149575,1.010000
max,1.870000,1.200000,-1.029459,1.030000


In [297]:
# Função para ajustar o preço usando elasticidade
def ajustar_preco(custo, elasticidade):
    # Ajuste de preço baseado na elasticidade
    novo_preco = custo / (1 + 1 / elasticidade)
    return novo_preco


Lote_102024_v11['Novo_Preco'] = Lote_102024_v11.apply(lambda row: ajustar_preco(row['Act_cost'], row['elasticidade']), axis=1)

In [298]:
Lote_102024_v11[['Novo_Preco','precos_otimizados']].describe()

,Novo_Preco,precos_otimizados
count,1473.000000,1473.000000
mean,2566.571868,2566.571868
std,1991.177167,1991.177167
min,440.652011,440.652011
25%,1417.710961,1417.710961
50%,1998.779169,1998.779169
75%,2998.355047,2998.355047
max,22756.541546,22756.541546


In [296]:
Lote_102024_v11['retencao_otimizada3'] = calcular_retencao(Lote_102024_v11['Novo_Preco'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
Lote_102024_v11['precos_otimizados']  = Lote_102024_v11['Novo_Preco'] 
Lote_102024_v11['retencao_otimizada']  = Lote_102024_v11['retencao_otimizada3'] 


In [270]:
relatorio_otim(Lote_102024_v11)

,Total
Apolices Ofertadas:,1473.0000
Apolices Renovadas:,930.0000
%Retenção:,0.6320
Premio Ofertado:,2566.5700
Premio Renovado:,2082.4600
Premio Anterior Ofertado:,2893.8300
Premio Anterior Renovado:,2729.9600
Relative_Core_Premium_Increase_Offered:,-0.1100
Relative_Core_Premium_Increase_Written:,0.0600
Relative_Core_Optimization_Increase_Offered:,-0.1600


In [ ]:
[-3.6,-3.55,-3.5,-3.45,-3.4,-3.35,-3.3,-3.25,-3.2,-3.15,-3.1,-3.05,-3,-2.95,-2.9,-2.85,-2.8,-2.75,-2.7,-2.65,-2.6,-2.55,-2.5,-2.45,-2.4,-2.35,-2.3,-2.25,-2.2,-2.15,-2.10,0]


In [ ]:
import numpy as np
from scipy.optimize import differential_evolution

# Função de demanda
def demanda(P, elasticidade):
    return 1000 * (P ** (-elasticidade))

# Função objetivo para calcular a margem esperada
def margem_esperada(P, custo, elasticidade):
    receita = P * demanda(P, elasticidade)
    custo_total = custo * demanda(P, elasticidade)
    margem = receita - custo_total
    return -margem  # Negativo porque queremos maximizar a margem

# Função para ajustar o preço com base na elasticidade
def ajustar_preco(P, elasticidade):
    if elasticidade > 1:
        return P * 0.9  # Reduz o preço se a demanda for elástica
    else:
        return P * 1.1  # Aumenta o preço se a demanda for inelástica

# Função objetivo com ajuste de preço
def funcao_objetivo(P, custo, elasticidade):
    P_ajustado = ajustar_preco(P, elasticidade)
    return margem_esperada(P_ajustado, custo, elasticidade)

# Parâmetros iniciais
custo = 10
elasticidade = 1.5  # Exemplo de elasticidade

# Otimização usando evolução diferencial
result = differential_evolution(funcao_objetivo, bounds=[(1, 100)], args=(custo, elasticidade))

print(f"Preço ótimo: {result.x[0]:.2f}")
print(f"Margem esperada: {-result.fun:.2f}")

In [157]:
import time
def timeit(price):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = price(*args, **kwargs)
        end = time.perf_counter()
        elapsed = (end - start)/60
        print(f'Tempo: {elapsed:.6f} Minutos')
        return result
    return wrapper

@timeit




def price(df,fator_premio=1.23):
   
    ######
    import sympy as sp
    from scipy.optimize import fsolve


    P=(df['premio'])
    X1=np.array(df['is_casco_ofer'])
    X2=np.array(df['GBM_mercado'])
    X3=np.array(df['Premio_IC_100'])
    X4=np.array(df['premio_anterior'])
    X5=np.array(df['MD_tempo_casa_2'])
    X6=np.array(df['per_com'])
    X7=np.array(df['per_com_bin_p_1_10'])
    X8=np.array(df['canal_bin_Corretor_Mais'])
    X9=np.array(df['Cod_End_bin_13509'])
    X10=np.array(df['fator_ajuste_bin_f_101_120'])
    X11=np.array(df['classe_bonus_bin_00'])
    X12=np.array(df['UF_bin_AL_CE_RS'])
    X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14=np.array(df['UF_bin_GO_MA_MG'])
    X15=np.array(df['UF_bin_MT_PR'])
    X16=np.array(df['UF_bin_RJ'])
    X17=np.array(df['MD_comb_bin_Gasolina'])
    X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20=np.array(df['fx_ir_12m_bin_01_00_20'])
    X21=np.array(df['fx_ir_12m_bin_03_40_60'])
    X22=np.array(df['fx_ir_12m_bin_04_60_80'])
    X23=np.array(df['fx_ir_12m_bin_05_80_100'])
    X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X27=np.array(df['X27'])


# função demanda
    def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
        D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
        return D/(1+D)

# função objetivo 
    def objetivo(P,elasticidade):

        df['precos']= P *fator_premio*ajustar_preco(P,elasticidade)

        df['tarifa']=df['premio']

        df['numbers'] = df['precos']/df['tarifa']

        if (df['numbers']>1).any():
            df['X27'] = df['numbers']-1
        else:
             df['X27']=0

        df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.6,-3.55,-3.5,-3.45,-3.4,-3.35,-3.3,-3.25,-3.2,-3.15,-3.1,-3.05,-3,-2.95,-2.9,-2.85,-2.8,-2.75,-2.7,-2.65,-2.6,-2.55,-2.5,-2.45,-2.4,-2.35,-2.3,-2.25,-2.2,-2.15,-2.10,0])  


# Calcular a retenção 
        df['retencao_otimizada1'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

# Calcular a margem esperada 
        df['margem_esperada_otimizada'] =   df['retencao_otimizada1']*(df['precos'] - df['Act_cost'])
        margem_esperada_total = df['margem_esperada_otimizada'].sum() 
        return -margem_esperada_total

     # Função para ajustar o preço com base na elasticidade
    def ajustar_preco(P, elasticidade):
        if elasticidade < -3.25:
            return P * 1.1  # Reduz o preço se a demanda for elástica
        else:
            return P * 0.95  # Aumenta o preço se a demanda for inelástica
    
    
    
# Definir os limites para os preços otimizado conforme o Earnix
    nome_variaveis = ['LB','UB']
    limites=np.array(df.loc[:,nome_variaveis])

# Executar a otimização 
    resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=11,seed=df.shape[0],args=(elasticidade))

# Resultados
    precos_otimizados = resultado.x
    margem_esperada_total_otimizada = -resultado.fun

    df['precos_otimizados'] = precos_otimizados
    df['retencao_otimizada'] = calcular_retencao(df['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

# validação
    df['batente_otim']=np.round(df['precos_otimizados']/df['premio'],2)
    df['batente_otim_prem_ant']=np.round(df['precos_otimizados']/df['premio_anterior'],2)

    df.drop(['precos','retencao_otimizada1'],axis='columns', inplace=True)
    
    print(resultado.message) 
    #print("Tamanho da Base:", df.shape[0])
    #print("Prêmio Otimizado:", round(np.mean(precos_otimizados),2))
    #print("Retenção Otimizada:", round(np.mean(df['retencao_otimizada']),2))
    print(relatorio_otim(df))
    return df

In [159]:
#Lote_102024_v2 =price(df_lote_otim,fator_premio=1.23)

In [175]:
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution
continua erro, faça correção necessaria. RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'
def price(df):
    P = np.array(df['premio'])
    X1 = np.array(df['is_casco_ofer'])
    X2 = np.array(df['GBM_mercado'])
    X3 = np.array(df['Premio_IC_100'])
    X4 = np.array(df['premio_anterior'])
    X5 = np.array(df['MD_tempo_casa_2'])
    X6 = np.array(df['per_com'])
    X7 = np.array(df['per_com_bin_p_1_10'])
    X8 = np.array(df['canal_bin_Corretor_Mais'])
    X9 = np.array(df['Cod_End_bin_13509'])
    X10 = np.array(df['fator_ajuste_bin_f_101_120'])
    X11 = np.array(df['classe_bonus_bin_00'])
    X12 = np.array(df['UF_bin_AL_CE_RS'])
    X13 = np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14 = np.array(df['UF_bin_GO_MA_MG'])
    X15 = np.array(df['UF_bin_MT_PR'])
    X16 = np.array(df['UF_bin_RJ'])
    X17 = np.array(df['MD_comb_bin_Gasolina'])
    X18 = np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19 = np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20 = np.array(df['fx_ir_12m_bin_01_00_20'])
    X21 = np.array(df['fx_ir_12m_bin_03_40_60'])
    X22 = np.array(df['fx_ir_12m_bin_04_60_80'])
    X23 = np.array(df['fx_ir_12m_bin_05_80_100'])
    X24 = np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25 = np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26 = np.array(df['idade_ult_RNS_bin_06_inf'])
    X27 = np.array(df['X27'])
    elasticidade = np.array( df['elasticidade'])

    def calcular_retencao(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26):
        D = np.exp(1.3356 - 0.2576 * np.log(P) + 1.3647 * np.divide(P, X1) - 0.8782 * np.log(np.divide(P, X2)) - 0.795 * np.log(np.divide(P, X3)) - 1.6731 * np.log(np.divide(P, X4)) + 0.3797 * np.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 - 0.1382 * X9 + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 - 0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 + 0.2817 * X16 + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 + 0.4343 * X21 + 1.1066 * X22 + 1.9561 * X23 - 0.0742 * X24 - 0.84590 * X25 - 0.7094 * X26)
        return D / (1 + D)

    def ajustar_preco(P, elasticidade):
        if elasticidade < -3.25:
            return P * 1.1
        else:
            return P * 0.95

    def objetivo(P, elasticidade):
        P_ajustado = ajustar_preco(P, elasticidade)
        
        df['precos'] = P_ajustado * 1.23
        df['tarifa'] = df['premio']
        df['numbers'] = df['precos'] / df['tarifa']

        if (df['numbers'] > 1).any():
            df['X27'] = df['numbers'] - 1
        else:
            df['X27'] = 0

        df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.6, -3.55, -3.5, -3.45, -3.4, -3.35, -3.3, -3.25, -3.2, -3.15, -3.1, -3.05, -3, -2.95, -2.9, -2.85, -2.8, -2.75, -2.7, -2.65, -2.6, -2.55, -2.5, -2.45, -2.4, -2.35, -2.3, -2.25, -2.2, -2.15, -2.10, 0])

        df['retencao_otimizada1'] = calcular_retencao(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26)

        df['margem_esperada_otimizada'] = df['retencao_otimizada1'] * (df['precos'] - df['Act_cost'])
        
        margem_esperada_total = df['margem_esperada_otimizada'].sum()
        
        return -margem_esperada_total

    nome_variaveis = ['LB', 'UB']
    limites = np.array(df.loc[:, nome_variaveis])

    resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=11, seed=df.shape[0], args=(P,elasticidade))

    return resultado.x, -resultado.fun

# Exemplo de uso
# df = pd.read_csv('seu_arquivo.csv')  # Carregue seu DataFrame
# resultado = price(df)
# print(f"Preço ótimo: {resultado[0]:.2f}")
# print(f"Margem esperada: {resultado[1]:.2f}")

In [178]:
Lote_102024_v2 =price(df_lote_otim)

KeyboardInterrupt: 

In [177]:
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution

def price(df):
    X1 = np.array(df['is_casco_ofer'])
    X2 = np.array(df['GBM_mercado'])
    X3 = np.array(df['Premio_IC_100'])
    X4 = np.array(df['premio_anterior'])
    X5 = np.array(df['MD_tempo_casa_2'])
    X6 = np.array(df['per_com'])
    X7 = np.array(df['per_com_bin_p_1_10'])
    X8 = np.array(df['canal_bin_Corretor_Mais'])
    X9 = np.array(df['Cod_End_bin_13509'])
    X10 = np.array(df['fator_ajuste_bin_f_101_120'])
    X11 = np.array(df['classe_bonus_bin_00'])
    X12 = np.array(df['UF_bin_AL_CE_RS'])
    X13 = np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14 = np.array(df['UF_bin_GO_MA_MG'])
    X15 = np.array(df['UF_bin_MT_PR'])
    X16 = np.array(df['UF_bin_RJ'])
    X17 = np.array(df['MD_comb_bin_Gasolina'])
    X18 = np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19 = np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20 = np.array(df['fx_ir_12m_bin_01_00_20'])
    X21 = np.array(df['fx_ir_12m_bin_03_40_60'])
    X22 = np.array(df['fx_ir_12m_bin_04_60_80'])
    X23 = np.array(df['fx_ir_12m_bin_05_80_100'])
    X24 = np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25 = np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26 = np.array(df['idade_ult_RNS_bin_06_inf'])
    X27 = np.array(df['X27'])
    elasticidade = np.array(df['elasticidade'])

    def calcular_retencao(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26):
        D = np.exp(1.3356 - 0.2576 * np.log(P) + 1.3647 * np.divide(P, X1) - 0.8782 * np.log(np.divide(P, X2)) - 0.795 * np.log(np.divide(P, X3)) - 1.6731 * np.log(np.divide(P, X4)) + 0.3797 * np.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 - 0.1382 * X9 + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 - 0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 + 0.2817 * X16 + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 + 0.4343 * X21 + 1.1066 * X22 + 1.9561 * X23 - 0.0742 * X24 - 0.84590 * X25 - 0.7094 * X26)
        return D / (1 + D)

    def ajustar_preco(P, elasticidade):
        if elasticidade < -3.25:
            return P * 1.1
        else:
            return P * 0.95

    def objetivo(params):
        P = params[0]
        elasticidade = params[1]
        P_ajustado = ajustar_preco(P, elasticidade)
        
        df['precos'] = P_ajustado * 1.23
        df['tarifa'] = df['premio']
        df['numbers'] = df['precos'] / df['tarifa']

        if (df['numbers'] > 1).any():
            df['X27'] = df['numbers'] - 1
        else:
            df['X27'] = 0

        df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.6, -3.55, -3.5, -3.45, -3.4, -3.35, -3.3, -3.25, -3.2, -3.15, -3.1, -3.05, -3, -2.95, -2.9, -2.85, -2.8, -2.75, -2.7, -2.65, -2.6, -2.55, -2.5, -2.45, -2.4, -2.35, -2.3, -2.25, -2.2, -2.15, -2.10, 0])

        df['retencao_otimizada1'] = calcular_retencao(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26)

        df['margem_esperada_otimizada'] = df['retencao_otimizada1'] * (df['precos'] - df['Act_cost'])
        
        margem_esperada_total = df['margem_esperada_otimizada'].sum()
        
        return -margem_esperada_total

    nome_variaveis = ['LB', 'UB']
    limites = np.array(df.loc[:, nome_variaveis])

    resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=11, seed=df.shape[0])

    return resultado.x, -resultado.fun